In [ ]:
import pandas as pd
import numpy as np

import requests
import os
import pandas as pd
import numpy as np
import nltk
pd.options.display.max_columns
pd.set_option("display.max_columns",100)

## Read Articles

In [ ]:
df_news = pd.read_csv("../NEW_DATA/PENDING/nafteboriki_ellada_fb1.csv")

In [ ]:
df_news.columns

In [ ]:
df_news

## Count Words

In [ ]:
#df_letter['num_wds'] = df_letter['tokenized'].apply(lambda x: len(x.split()))
df_news['num_wds'] = df_news['article_text'].apply(lambda x: len(x.split()) if pd.notna(x) and type(x) == str else 0)
df_news['num_wds'].mean()

In [ ]:
df_news['num_wds'].describe()

In [ ]:
ax=df_news['num_wds'].plot(kind='hist', bins=5, fontsize=14, figsize=(6,5))
ax.set_title('Text length\n', fontsize=10)
ax.set_ylabel('Frequency', fontsize=10)
ax.set_xlabel('Number of Words', fontsize=10)

## Drop under 100 words

In [ ]:
#drop rows with texts< 60 words
df_news = df_news[df_news['num_wds'] >= 60]

In [ ]:
df_news = df_news.reset_index()

In [ ]:
df_news

## Count Unique words

In [ ]:
df_news['uniq_wds'] = df_news['article_text'].apply(lambda x: len(set(x.split())) if pd.notna(x) and type(x) == str else 0)
df_news['uniq_wds'].head()

In [ ]:
ax=df_news['uniq_wds'].plot(kind='hist', bins=50, fontsize=14, figsize=(12,10))
ax.set_title('Unique Words Per Text\n', fontsize=20)
ax.set_ylabel('Frequency', fontsize=18)
ax.set_xlabel('Number of Unique Words', fontsize=18);

## Lexical Richness

In [ ]:
from collections import Counter
def calculate_mtld(text, chunk_size=100):
    tokens = text.split()  # Tokenize the text (simple splitting by whitespace)

    unique_words = set()  # Track unique words
    mtld_count = 0
    total_tokens = len(tokens)

    for i, token in enumerate(tokens):
        unique_words.add(token)

        if len(unique_words) >= chunk_size:
            mtld_count += 1
            unique_words.clear()

    # Calculate MTLD score
    mtld_score = total_tokens / (mtld_count + 1)

    return mtld_score

In [ ]:
df_news['MTLD'] = df_news['article_text'].apply(lambda x: calculate_mtld(x))

## COUNT NOUNS VERBS ADJECTIVES

In [ ]:
import spacy
# from spacy import displacy 
from spacy.matcher import Matcher
import pandas as pd
import unicodedata
from unidecode import unidecode
from Levenshtein import *
from collections import defaultdict
import textdescriptives as td  #TEXTDESCRIPTIVES FR PSACY3
import numpy as np
nlp = spacy.load("el_core_news_lg")

In [ ]:
# Function to process text and count nouns, verbs, and adjectives
def find_nouns_verbs_adjectives(text):
    doc = nlp(text)
    nouns = sum(1 for token in doc if token.pos_ == "NOUN")
    verbs = sum(1 for token in doc if token.pos_ == "VERB")
    adjectives = sum(1 for token in doc if token.pos_ == "ADJ")
    return nouns, verbs, adjectives

In [ ]:
# Apply the function to each row of the DataFrame
df_news[["Nouns", "Verbs", "Adjectives"]] = df_news["article_text"].apply(lambda x: pd.Series(find_nouns_verbs_adjectives(x)))

In [ ]:
df_news["Nouns"] = df_news["Nouns"]/ df_news['num_wds'] *100

In [ ]:
df_news["Verbs"] = df_news["Verbs"]/ df_news['num_wds'] *100

In [ ]:
df_news["Adjectives"] = df_news["Adjectives"]/ df_news['num_wds'] *100

In [ ]:
df_news.head(2)

## Intensity Lexcicon

In [ ]:
AIL_df = pd.read_csv("../LEXIKA/AIL_KS.csv")

In [ ]:
import nltk.sentiment
w2affect = []
for w,score,dim in AIL_df.values:
    w2affect.append({dim:score,'w':w})
w2affects = {}
df = pd.DataFrame(w2affect)
df = df.groupby('w').sum().reset_index()
for w,anger,fear,joy,sadness, trust, disgust, anticipation, surprise in df[['w','anger','fear','joy','sadness', 'trust', 'disgust', 'anticipation', 'surprise']].values:
    w2affects[w] = {'anger':anger,'joy':joy,'sadness':sadness,'fear':fear,'trust': trust, 'disgust':disgust, 'anticipation':anticipation,'surprise': surprise }

def get_affect_intensity_score(doc,tokenizer=nltk.word_tokenize,agg='mean'):
    if type(doc)==str:
        doc = tokenizer(doc)
    assert type(doc)==list,"please input either a list or a string"
    matches = []
    for w in doc:
        w = w.lower()
        if w in w2affects:
            matches.append(w2affects[w])
    if len(matches)==0:
        return {'anger':np.nan,'joy':np.nan,'sadness':np.nan,'fear':np.nan,'trust':np.nan,'disgust':np.nan,'anticipation':np.nan,'surprise':np.nan}
    scores = pd.DataFrame(matches)
    if agg=='mean':
        scores = scores.mean()
    elif agg=='max':
        scores =  scores.max()
    else:
        scores = agg(scores)
    return dict(scores)

get_affect_intensity_score('Υπάρχει πολύ μίσος και αηδία στην ατμόσφαιρα. Είσαι απαίσιος!')

In [ ]:
# only anger 

w2affect = []
for w,score,dim in AIL_df.values:
    w2affect.append({dim:score,'w':w})
w2affects = {}
df = pd.DataFrame(w2affect)
df = df.groupby('w').sum().reset_index()
for w,anger,fear,joy,sadness in df[['w','anger','fear','joy','sadness']].values:
    w2affects[w] = {'anger':anger,'joy':joy,'sadness':sadness,'fear':fear}

def get_anger_score(doc,tokenizer=nltk.word_tokenize,agg='mean'):
    if type(doc)==str:
        doc = tokenizer(doc)
    assert type(doc)==list,"please input either a list or a string"
    matches = []
    for w in doc:
        w = w.lower()
        if w in w2affects:
            matches.append(w2affects[w])
    if len(matches)==0:
        return {'anger':np.nan,'joy':np.nan,'sadness':np.nan,'fear':np.nan}
    scores = pd.DataFrame(matches)
    if agg=='mean':
        scores = scores.mean()
    elif agg=='max':
        scores =  scores.max()
    else:
        scores = agg(scores)
    return list(dict(scores).values())[0]

get_anger_score('Είμαι πολύ θυμωμένος και έχω νεύρα και θυμό και δεν αντέχω!')

In [ ]:
df_news ['Anger_intensity'] = df_news['cleaned_text_lemmas'].apply(lambda x: get_anger_score(x))

In [ ]:
# only joy

w2affect = []
for w,score,dim in AIL_df.values:
    w2affect.append({dim:score,'w':w})
w2affects = {}
df = pd.DataFrame(w2affect)
df = df.groupby('w').sum().reset_index()
for w,anger,fear,joy,sadness in df[['w','anger','fear','joy','sadness']].values:
    w2affects[w] = {'anger':anger,'joy':joy,'sadness':sadness,'fear':fear}

def get_joy_score(doc,tokenizer=nltk.word_tokenize,agg='mean'):
    if type(doc)==str:
        doc = tokenizer(doc)
    assert type(doc)==list,"please input either a list or a string"
    matches = []
    for w in doc:
        w = w.lower()
        if w in w2affects:
            matches.append(w2affects[w])
    if len(matches)==0:
        return {'anger':np.nan,'joy':np.nan,'sadness':np.nan,'fear':np.nan}
    scores = pd.DataFrame(matches)
    if agg=='mean':
        scores = scores.mean()
    elif agg=='max':
        scores =  scores.max()
    else:
        scores = agg(scores)
    return list(dict(scores).values())[1]

get_joy_score('είσαι ωραίος και χαίρομαι να σε βλέπω.')

In [ ]:
df_news ['Joy_intensity'] = df_news['cleaned_text_lemmas'].apply(lambda x: get_joy_score(x))

In [ ]:
# only sadness

w2affect = []
for w,score,dim in AIL_df.values:
    w2affect.append({dim:score,'w':w})
w2affects = {}
df = pd.DataFrame(w2affect)
df = df.groupby('w').sum().reset_index()
for w,anger,fear,joy,sadness in df[['w','anger','fear','joy','sadness']].values:
    w2affects[w] = {'anger':anger,'joy':joy,'sadness':sadness,'fear':fear}

def get_sadness_score(doc,tokenizer=nltk.word_tokenize,agg='mean'):
    if type(doc)==str:
        doc = tokenizer(doc)
    assert type(doc)==list,"please input either a list or a string"
    matches = []
    for w in doc:
        w = w.lower()
        if w in w2affects:
            matches.append(w2affects[w])
    if len(matches)==0:
        return {'anger':np.nan,'joy':np.nan,'sadness':np.nan,'fear':np.nan}
    scores = pd.DataFrame(matches)
    if agg=='mean':
        scores = scores.mean()
    elif agg=='max':
        scores =  scores.max()
    else:
        scores = agg(scores)
    return list(dict(scores).values())[2]

get_sadness_score('Δεν μπορεί να ζούμε αυτή την τραγωδία που έπληξε τη χώρα. ')



In [ ]:
df_news ['Sadness_intensity'] = df_news['cleaned_text_lemmas'].apply(lambda x: get_sadness_score(x))

In [ ]:
# only fear

w2affect = []
for w,score,dim in AIL_df.values:
    w2affect.append({dim:score,'w':w})
w2affects = {}
df = pd.DataFrame(w2affect)
df = df.groupby('w').sum().reset_index()
for w,anger,fear,joy,sadness in df[['w','anger','fear','joy','sadness']].values:
    w2affects[w] = {'anger':anger,'joy':joy,'sadness':sadness,'fear':fear}

def get_fear_score(doc,tokenizer=nltk.word_tokenize,agg='mean'):
    if type(doc)==str:
        doc = tokenizer(doc)
    assert type(doc)==list,"please input either a list or a string"
    matches = []
    for w in doc:
        w = w.lower()
        if w in w2affects:
            matches.append(w2affects[w])
    if len(matches)==0:
        return {'anger':np.nan,'joy':np.nan,'sadness':np.nan,'fear':np.nan}
    scores = pd.DataFrame(matches)
    if agg=='mean':
        scores = scores.mean()
    elif agg=='max':
        scores =  scores.max()
    else:
        scores = agg(scores)
    return list(dict(scores).values())[3]

get_fear_score('Είναι φοβερή η τραγωδία που έπληξε τη χώρα. ')

In [ ]:
df_news ['Fear_intensity'] = df_news['cleaned_text_lemmas'].apply(lambda x: get_fear_score(x))

In [ ]:
df_news [['Anger_intensity','Joy_intensity', 'Sadness_intensity','Fear_intensity']]

In [ ]:
cols = ['Anger_intensity', 'Joy_intensity', 'Sadness_intensity',
       'Fear_intensity']
df_news[cols] = df_news[cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [ ]:
df_news.head(2)

In [ ]:
df_news['joy']

In [ ]:
df_news['joy']

In [ ]:
df_news = df_news.rename(columns={' anticipation': 'anticipation','num_wds': 'length'})

In [ ]:
df_news['disgust'] =  df_news['disgust']/ df_news['uniq_wds'] *100
df_news['fear'] =  df_news['fear']/ df_news['uniq_wds'] *100
df_news['surprise'] =  df_news['surprise']/ df_news['uniq_wds'] *100
df_news['trust'] =  df_news['trust']/ df_news['uniq_wds'] *100
df_news['sadness'] =  df_news['sadness']/ df_news['uniq_wds'] *100
df_news['negative'] =  df_news['negative']/ df_news['uniq_wds'] *100
df_news['joy'] =  df_news['joy']/ df_news['uniq_wds'] *100
df_news['positive'] =  df_news['positive']/ df_news['uniq_wds'] *100
df_news['anticipation'] =  df_news['anticipation']/ df_news['uniq_wds'] *100
df_news['anger'] =  df_news['anger']/ df_news['uniq_wds'] *100

In [ ]:
df_news.tail(2)

In [ ]:
df_news['strongsubj'] =  df_news['strongsubj']/ df_news['uniq_wds'] *100

In [ ]:
df_news['weaksubj'] =  df_news['weaksubj']/ df_news['uniq_wds'] *100

## Count celebrities etc

In [ ]:
# ιδανικά ένα if ent.label_ == "PERSON" ώστε να κραταει μονο τα persons όχι και τα ονόματα οδών 

#def get_entities(doc):
#    entity_info = [(ent.text, ent.label_) for ent in doc.ents]
#    return entity_info

In [ ]:
import eli5

In [ ]:
import graphviz

In [ ]:
def get_entities(text):
    doc = nlp(text)
    entity_info = [(ent.text) for ent in doc.ents if (ent.label_ == 'PERSON')]
    return entity_info

In [ ]:
doc = nlp("Ήρθε η Αννα Βισση και ο Σκοιχάς και ο βατραχος και η μηχανη και η Microsoft και η Ελλάδα")
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
get_entities(doc)

In [ ]:
df_news['names'] = df_news['article_text'].apply(lambda x: get_entities(x))

In [ ]:
df_news['names'] = df_news['names'].apply(', '.join)

In [ ]:
def drop_duplicates(row):
    # Split string by ', ', drop duplicates and join back.
    words = row.split(', ')
    return ', '.join(np.unique(words).tolist())

# drop_duplicates is applied to all rows of df.
df_news['names'] = df_news['names'].apply(drop_duplicates)

In [ ]:
df_news['names']

In [ ]:
politicians = pd.read_csv('../TO_KEEP/politiciansel.csv')
politicians.head()

In [ ]:
politicians_list = politicians['Politician'].tolist()
politicians_list

In [ ]:
celebrities = pd.read_csv('../TO_KEEP/celebrities')
celebrities.head()

In [ ]:
[celebrities['celebrities'].unique() for col_name in celebrities.columns]

In [ ]:
celebrities_list = celebrities['celebrities'].tolist()
celebrities_list

In [ ]:
joinedlist = celebrities_list + politicians_list

In [ ]:
def find_celebs(row):
    celebrities = []
    # Split string by ', ', drop duplicates and join back.
    words = row.split(', ')
    for i in words:
        if i in joinedlist:
            
            celebrities.append(i)
    return celebrities
                


# drop_duplicates is applied to all rows of df.
df_news['celebrities'] = df_news['names'].apply(find_celebs)

In [ ]:
df_news['celebrities']

In [ ]:
def count_celebs(row):
    celebrities = []
    count  =0
    # Split string by ', ', drop duplicates and join back.
    words = row.split(', ')
    for i in words:
        
        if i in joinedlist:
            
            celebrities.append(i)
            count +=1 
    return count
                


# drop_duplicates is applied to all rows of df.
df_news['No Celebs'] = df_news['names'].apply(count_celebs)

In [ ]:
df_news['No Celebs']

In [ ]:
df_news['No Celebs'].value_counts()

In [ ]:
animals = pd.read_csv('../TO_KEEP/animals')
animals.tail()

In [ ]:
animal_list = animals['ζώο'].tolist()
animal_list

In [ ]:
def find_animals(row):
    animals = []
    words = row.split(' ')
    for i in words:
        if i in animal_list:
            animals.append(i)
    return animals
                
df_news['animal'] = df_news['article_text'].apply(find_animals)

In [ ]:
df_news['animal'].value_counts().head(200)

In [ ]:
def count_animals(row):
    animals = []
    count  =0
    words = row.split(' ')
    for i in words:
        
        if i in animal_list:
            
            animals.append(i)
            count +=1 
    return count

df_news['animal'] = df_news['article_text'].apply(count_animals)

In [ ]:
crime = pd.read_csv('../TO_KEEP/crime.csv')
crime.tail()

In [ ]:
crime['crime'] = crime['crime'].str.lower()

In [ ]:
crime_list = crime['crime'].tolist()
crime_list

In [ ]:
def find_crime(row):
    crime = []
    # Split string by ', ', drop duplicates and join back.
    words = row.split(' ')
    #print(words)
    for i in words:
        if i in crime_list:
            
            crime.append(i)
    return crime
                


# drop_duplicates is applied to all rows of df.
df_news['crime'] = df_news['article_text'].apply(find_crime)

In [ ]:
df_news['crime'].value_counts()

In [ ]:
df_news['crime']

In [ ]:
def count_crime(row):
    crime = []
    count  =0
    words = row.split(' ')
    for i in words:
        
        if i in crime_list:
            
            crime.append(i)
            count +=1 
    return count
df_news['crime'] = df_news['article_text'].apply(count_crime)

In [ ]:
df_news['crime']

In [ ]:
sensual = pd.read_csv('../TO_KEEP/sensual.csv')
sensual.tail()

In [ ]:
sensual_list = sensual['sensual'].tolist()

In [ ]:
def find_sensual(row):
    sensual = []
    # Split string by ', ', drop duplicates and join back.
    words = row.split(' ')
    #print(words)
    for i in words:
        if i in sensual_list:
            
            sensual.append(i)
    return sensual
                


# drop_duplicates is applied to all rows of df.
df_news['sensual'] = df_news['article_text'].apply(find_sensual)

In [ ]:
def count_sensual(row):
    sensual = []
    count  =0
    # Split string by ', ', drop duplicates and join back.
    words = row.split(' ')
    for i in words:
        
        if i in sensual_list:
            
            sensual.append(i)
            count +=1 
    return count
                


# drop_duplicates is applied to all rows of df.
df_news['sensual'] = df_news['article_text'].apply(count_sensual)

In [ ]:
df_news['sensual'].value_counts()

In [ ]:
df_news

## Κεφαλαία στο κείμενο

In [ ]:
import re

regex = r'\b[A-Z]{5,}\b'

df_news['caps_body'] = df_news['article_text'].str.findall(regex)
df_news['caps_body'] =df_news['caps_body'].str.len()

In [ ]:
df_news['caps_body'].value_counts()

## Νούμερα 

In [ ]:
df_news['numbers'] = df_news['article_text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

In [ ]:
df_news['numbers']

In [ ]:
df_news['article_text'][2]

In [ ]:
## ίσως να μετρήσω προσωπικές αντωνυμίες

## Links

In [ ]:
import re

def find_links(text):
    urls = re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', text)
    return urls

def number_links(text):
    urls = re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', text)
    number = len(urls)
    return number

In [ ]:
df_news['urls'] = df_news.article_text.apply(find_links)

In [ ]:
df_news['urls'].value_counts()

In [ ]:
df_news['urls'] = df_news.article_text.apply(number_links)

# # Special Character Ratio

In [ ]:
def check_freq(x):
    freq = 0
    for c in ['"', '!', '?',':','«', '»', ';']:
       freq += x.count(c)
    return freq

In [ ]:
df_news ['special_char_ratio'] = (df_news.article_text.apply(check_freq) / df_news['uniq_wds']) * 100

In [ ]:
df_news['special_char_title'] = (df_news.article_title.apply(check_freq) / df_news['uniq_wds']) * 100

In [ ]:
#define subjectivity
def define_subjectivity(column):
    words = column.split()

    strongsubj = 0
    weaksubj = 0
    matched_tokens = 0

    for word in words:
        found_exact = False
        if word in indexes:
            found_exact = True
            indx = indexes[word]

            if df.at[indx, "Strength"] == "strongsubj":
                strongsubj = strongsubj + 1
            elif df.at[indx, "Strength"] == "weaksubj":
                weaksubj = weaksubj + 1

            matched_tokens+= 1
    try:
        return (strongsubj, weaksubj)

    except:
        return False
#Load Subjectivity vocabulary
df = pd.read_csv("../LEXIKA/Greek_Subjectivity_Fixed_2023_02.csv")
indexes = {}
df = df.fillna("N/A")
for index, row in df.iterrows():
    df.at[index, "Greek"] = row["Greek"]
    indexes[df.at[index, "Greek"]] = index
#calculate Subjectivity for lemmas
df_news[['strongsubjTitle','weaksubjTitle']] =  df_news['article_title'].apply(lambda x: pd.Series(define_subjectivity(x)))


In [ ]:
df_news['strongsubjTitle'] =  df_news['strongsubjTitle']/ df_news['uniq_wds'] *100

In [ ]:
df_news['WeakSTitle'] =  df_news['weaksubjTitle']/ df_news['uniq_wds'] *100

In [ ]:
df_news.head(3)

In [ ]:
df_news.to_csv("../NEW_DATA/PENDING/nafteboriki_ellada_fb1.csv", index = False)